In [2]:
import numpy as np
import os
import sys
sys.path.append(r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project')

import matplotlib.pyplot as plt # for plotting
%matplotlib widget
import pandas as pd # for data manipulation
from Models.load_data import *
from Models.helpers import *

%load_ext autoreload
%autoreload 2

import torch

path = r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Training_data'
features_list = os.listdir(os.path.join(path, r'features'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# This cell takes 15s
# Instantiate the dataset without transformations
dataset = IndependentCSVDataset(path, features_list)

# Split dataset into training and testing sets
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

feature_min, feature_max = compute_global_minmax(train_dataset)
print("Global feature min (train set only):", feature_min)
print("Global feature max (train set only):", feature_max)

global_transform = GlobalMinMaxNormalize(feature_min, feature_max)
train_dataset.dataset.transform = global_transform
test_dataset.dataset.transform = global_transform  # Same transform to prevent data leakage

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)



Skipping JETno87125.csv: sequence length 6001 is unexpected.
Skipping JETno91071.csv: sequence length 6001 is unexpected.
Skipping JETno97612.csv: sequence length 6001 is unexpected.
Skipping JETno99471.csv: sequence length 6001 is unexpected.
Global feature min (train set only): [ 0.0000000e+00 -3.5154782e+06 -1.0885503e+14  0.0000000e+00
 -1.2418192e+14  0.0000000e+00]
Global feature max (train set only): [6.8073063e+00 7.2445923e+03 6.6544650e+17 8.5139590e+17 1.8753850e+18
 1.6388709e+04]


In [4]:
# Create an iterator from the DataLoader
train_iterator = iter(train_loader)

# Get a single batch
batch = next(train_iterator)

# Unpack the batch (assuming your dataset returns (inputs, targets))
inputs, targets = batch

print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)


Inputs shape: torch.Size([32, 6000, 6])
Targets shape: torch.Size([32, 6000, 2])


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weights(train_loader)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

Class counts: [1395967.0, 20033.0]
Class weights: [0.014147598296403885, 0.9858523607254028]


In [ ]:

import torch.nn.functional as F  # Import for one-hot encoding

y_file = pd.read_csv(target_file)['target'].to_numpy()
target = torch.tensor(y_file, dtype=torch.long, device=device)
target = F.one_hot(target, num_classes=2).float()

In [7]:

input_size = 6       # Number of features per time step
hidden_size = 64     # Number of LSTM hidden units
num_layers = 2       # Number of LSTM layers
output_size = 2      # Binary classification
dropout = 0.2

# Instantiate the model
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)

# Print model architecture
print(model)

# Example dummy input: batch_size=32, sequence_length=6000, input_size=6
dummy_input = inputs
dummy_output = model(dummy_input)

print("Output shape:", dummy_output[0].shape)  # Expected: (32, 6000, 2) -> one binary value per timestep

loss = 0
for output, target in zip(dummy_output, targets):
    loss += criterion(output, target)

print(loss.item())

LSTMModel(
  (lstm): LSTM(6, 64, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=64, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)
Output shape: torch.Size([6000, 2])
1.0850722789764404


In [11]:
output_1 = torch.tensor([1.0, 0])
output_2 = torch.tensor([0, 1.0])

target_1 = torch.tensor([1.0, 0])
target_2 = torch.tensor([0, 1.0])

print(criterion(output_1, target_1), criterion(output_2, target_1))
print(criterion(output_1, target_2), criterion(output_2, target_2))

tensor(0.0044) tensor(0.0186)
tensor(1.2947) tensor(0.3088)


In [8]:
# With 10 rounds of optimisation, this tkes roughly 30 min on my laptop
import torch
import torch.optim as optim

# Define loss and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weights(train_loader)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))  # For regression; use CrossEntropyLoss for classification
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Assume `train_loader` is a DataLoader yielding batches of (input, target)
num_epochs = 1

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # inputs should have shape: [batch_size, seq_length, input_size]
        # targets should have shape: [batch_size, output_size] or [batch_size] (depending on your problem)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)

        # Manually sum up losses from batches, annoying but necessary for structure
        loss = torch.tensor(0.0, device=device, requires_grad=True)

        # Loop through the whole batch
        for output, target in zip(outputs, targets):
            batch_loss = criterion(output, target)  # Compute loss for this sample
            loss = loss + batch_loss  # Accumulate loss correctly
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")


Class counts: [1393760.0, 22240.0]
Class weights: [0.015706215053796768, 0.9842938184738159]


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)


Epoch [1/1], Loss: 0.6328


In [9]:
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score, roc_auc_score

def evaluate_test_metrics(model, test_loader, criterion, device, threshold=0.5):
    """
    Evaluates the test loss, F1-score, and ROC-AUC of a trained model.

    Parameters:
    - model: The trained PyTorch model.
    - test_loader: DataLoader for the test dataset.
    - criterion: Loss function (e.g., nn.CrossEntropyLoss, nn.BCELoss).
    - device: The device ('cuda' or 'cpu').
    - threshold: Decision threshold for binary classification (default=0.5).

    Returns:
    - Average test loss
    - F1-score
    - ROC-AUC score
    """
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    total_samples = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to device

            # Forward pass
            outputs = model(inputs)  # (batch_size, 6000, 2) -> logits

            # Compute loss for batch
            loss = criterion(outputs.view(-1, 2), targets.view(-1, 2))  # Flatten for loss function
            test_loss += loss.item() * inputs.size(0)
            total_samples += inputs.size(0)

            # Convert model outputs (logits) to probabilities
            probs = F.softmax(outputs, dim=-1)  # (batch_size, 6000, 2)

            # Get predicted class labels
            preds = torch.argmax(probs, dim=-1)  # (batch_size, 6000) -> class indices

            # Convert one-hot targets to class indices
            if targets.ndim > 2 and targets.shape[-1] > 1:  # One-hot case
                targets = torch.argmax(targets, dim=-1)  # (batch_size, 6000) -> class indices

            # Flatten for metric calculations
            all_targets.extend(targets.cpu().numpy().flatten())  # Convert to 1D list
            all_predictions.extend(preds.cpu().numpy().flatten())  # Convert to 1D list

    # Compute average loss
    avg_test_loss = test_loss / total_samples

    # Compute F1-score (use 'macro' for balanced classes)
    f1 = f1_score(all_targets, all_predictions, average='macro')

    # Compute ROC-AUC Score
    auc = roc_auc_score(all_targets, all_predictions)

    return avg_test_loss, f1, auc

# Example Usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loss, test_f1, auc = evaluate_test_metrics(model, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"AUC-ROC: {auc:.4f}")


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)


Test Loss: 0.0148
F1 Score: 0.0058
AUC-ROC: 0.5000
